# Часть 1. Проверка гипотезы в Python и составление аналитической записки

Гипотеза: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы.

Нулевая гипотеза $H_0: \mu_{\text{СПб}} \leq \mu_{\text{Москва}}$ <br> Среднее время активности пользователей в Санкт-Петербурге не больше, чем в Москве.

Альтернативная гипотеза $H_1: \mu_{\text{СПб}} > \mu_{\text{Москва}}$ <br> Среднее время активности пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

## Продолжительность активности пользователей из Санкт-Петербурга и Москвы 
- Автор: Сабурова Кристина
- Дата: 05.06.2025

## Цели и задачи проекта

### Цели
Оценить, существует ли статистически значимое различие в средней продолжительности активности пользователей из Санкт-Петербурга и Москвы в приложении для чтения и прослушивания книг

### Задачи
- Загрузить и проверить данные пользователей из файла yandex_knigi_data.csv.
- Проверить наличие дубликатов по user_id.
- Сравнить размеры групп по городам, рассчитать средние значения и визуализировать распределения.
- Провести односторонний t-тест для сравнения средней активности.
- Сделать выводы на основе значения p-value.
- Предложить возможные объяснения выявленных результатов.

## Описание данных

Для анализа были использованы данные из файла yandex_knigi_data.csv, содержащего информацию об активности пользователей в приложении для чтения и прослушивания книг
- `city` - Город проживания пользователя (Москва или Санкт-Петербург)
- `puid` - Уникальный идентификатор пользователя
- `hours` - Суммарное время активности пользователя в приложении (в часах)

## Содержимое проекта

- Постановка задачи и цели исследования
- Описание данных
- Предварительный анализ данных
- Проверка гипотезы
- Составление аналитической записки

---

## 1. Загрузка данных и знакомство с ними


In [ ]:
# Импортируем библиотеку и выгружаем данные
import pandas as pd
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_knigi_data.csv')

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
display(df.head())
display(df. info())

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


None

## 2. Проверка гипотезы в Python

Гипотеза: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. 

- H₀ (нулевая): Средняя активность пользователей из Санкт-Петербурга меньше или равна, чем у пользователей из Москвы.
- H₁ (альтернативная): Средняя активность пользователей из Санкт-Петербурга больше, чем у пользователей из Москвы.

In [ ]:
from scipy import stats as st

In [ ]:
# Отбираем пользователей по городам
moscow = df[df['city'] == 'Москва']['hours']
spb = df[df['city'] == 'Санкт-Петербург']['hours']

In [ ]:
# Посчитаем размеры выборок

print(f"Размер группы Москва: {len(moscow)}")
print(f"Размер группы СПб: {len(spb)}")

Размер группы Москва: 6234
Размер группы СПб: 2550


In [ ]:
# Посчитаем основные статистики

print("Москва — среднее:", moscow.mean(), "медиана:", moscow.median(), "стандартное отклонение:", moscow.std())
print("СПб — среднее:", spb.mean(), "медиана:", spb.median(), "стандартное отклонение:", spb.std())

Москва — среднее: 10.88109206345796 медиана: 0.9244980209724304 стандартное отклонение: 36.851682954030586
СПб — среднее: 11.59269077702618 медиана: 0.9847812626262626 стандартное отклонение: 39.704992814303964


Выводы по результатам анализа размера групп и их статистик
- Размеры выборок разные: в Москве пользователей больше, чем в Санкт-Петербурге
- Во всех группах наблюдается значительное расхождение между средним и медианой, а также большое стандартное отклонение
- Это говорит о наличии выбросов, а также о смещенном распределении данных
- В такой ситуации классический t-тест может быть неустойчив к искажению результатов, проведем тест Манна-Уитни

**Обоснование выбора теста**

Для проверки гипотезы использован односторонний t-тест для независимых выборок. Он подходит, потому что:

- сравниваются две независимые группы (Москва и Санкт-Петербург);
- анализируется среднее время активности — количественный показатель;
- гипотеза направленная: ожидается, что пользователи из СПб проводят больше времени, поэтому alternative='greater';
- уровень значимости — 0.05

In [ ]:
# Односторонний t-тест с поправкой Уэлча (equal_var=False)
stat, p_value = ttest_ind(spb, moscow, equal_var=False, alternative='greater')

print(f"t-статистика: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Отклоняем нулевую гипотезу: пользователи из СПб проводят статистически значимо больше времени в приложении.")
else:
    print("Нет оснований отклонить нулевую гипотезу: данные не подтверждают, что пользователи из СПб более активны.")

t-статистика: 0.7782
p-value: 0.2182
Нет оснований отклонить нулевую гипотезу: данные не подтверждают, что пользователи из СПб более активны.


In [ ]:
from scipy.stats import mannwhitneyu

# Выполняем тест Манна-Уитни
stat, p_value = mannwhitneyu(spb, moscow, alternative='greater')

print(f"U-статистика: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Интерпретация
if p_value < 0.05:
    print("Отклоняем нулевую гипотезу: пользователи из Санкт-Петербурга статистически значимо активнее.")
else:
    print("Нет оснований отклонить нулевую гипотезу: данные не подтверждают, что пользователи из Санкт-Петербурга более активны.")

U-статистика: 8093616.0000
p-value: 0.0891
Нет оснований отклонить нулевую гипотезу: данные не подтверждают, что пользователи из Санкт-Петербурга более активны.


## 3. Аналитическая записка



- Тип теста: Односторонний t-тест с поправкой Уэлча и тест Манна-Уитни. Уровень статистической значимости (α): 0.05
- p-value: 0.2182
- Поскольку p-value больше уровня значимости (0.2182 > 0.05), нет оснований отклонить нулевую гипотезу. Это означает, что статистически значимых различий в средней активности пользователей из Москвы и Санкт-Петербурга не выявлено
- Результаты могут быть связаны с тем, что: 
  - пользователи из обоих городов используют приложение с одинаковой частотой и продолжительностью
  - есть ограничения данных (возможна нехватка объёма выборки)

----

# Часть 2. Анализ результатов A/B-тестирования

## 1. Описание целей исследования.



Целью исследования является оценка эффективности нового интерфейса интернет-магазина BitMotion Kit, протестированного в формате A/B-теста.

Гипотеза теста: упрощение интерфейса повысит конверсию зарегистрированных пользователей в покупку как минимум на 3 процентных пункта в течение первых 7 дней после регистрации.

## 2. Загрузка данных, оценка целостности.


In [ ]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [ ]:
# Информация о данных участников 
display(participants.head())
display(participants. info())
# Информация о данных событий
display(events.head())
display(events.info())

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


None

,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


None

In [ ]:
# Проверим пропуски
print(participants.isnull().sum())

user_id    0
group      0
ab_test    0
device     0
dtype: int64


In [ ]:
# Проверим пропуски
print(events.isnull().sum())

user_id            0
event_dt           0
event_name         0
details       538264
dtype: int64


In [ ]:
# Удалим строки GLOBAL в user_id, так как они не имеют отношения к поведению конкретного пользователя и не участвуют в A/B-тесте
events = events[events['user_id'] != 'GLOBAL']

In [ ]:
# Проверим количество дубликатов по событиям
duplicates = events.duplicated().sum()
print(f"Количество дубликатов в events: {duplicates}")

Количество дубликатов в events: 35473


In [ ]:
# Удалим дубликаты
events = events.drop_duplicates()

## 3. По таблице `ab_test_participants` оценим корректность проведения теста:

In [ ]:
# Оставим только нужный тест
participants_eu = participants[participants['ab_test'] == 'interface_eu_test']

# Проверим соответствие ТЗ: только группы A и B
print(participants_eu['group'].unique())

['B' 'A']


Присутствуют только группы A и B, все верно

In [ ]:
# Проверим датафрейм на наличие неявных дубликатов в user_id
duplicates_id = participants['user_id'].duplicated().sum()

# Также рассчитаем долю дубликатов
share_id = duplicates_id / participants.shape[0] * 100

display(duplicates_id)
display(share_id)

887

6.10671256454389

In [ ]:
# Проверим равномерность распределения по группам
group_counts = participants_eu['group'].value_counts(normalize=True)
print(group_counts)

B    0.503871
A    0.496129
Name: group, dtype: float64


Группы распределены примерно 50/50 - хорошо

In [ ]:
# Оставим пользователей из группы B
group_b = participants[participants['group'] == 'B']

# Найдём пользователей, попавших в группу B более чем в одном тесте
b_multi_tests = group_b.groupby('user_id')['ab_test'].nunique()
b_problem_users = b_multi_tests[b_multi_tests > 1].index

print(f"Количество пользователей в группе B более чем в одном тесте: {len(b_problem_users)}")

# Удалим таких пользователей из выборки по нашему тесту
participants_eu_clean = participants_eu[~participants_eu['user_id'].isin(b_problem_users)]

print(f"Размер выборки после удаления проблемных пользователей: {participants_eu_clean.shape[0]}")

Количество пользователей в группе B более чем в одном тесте: 116
Размер выборки после удаления проблемных пользователей: 10734


In [ ]:
# Проверим пересечения пользователей между группами A и B внутри теста interface_eu_test
users_group_a = set(participants_eu[participants_eu['group'] == 'A']['user_id'])
users_group_b = set(participants_eu[participants_eu['group'] == 'B']['user_id'])

intersect_users = users_group_a.intersection(users_group_b)

print(f"Количество пользователей, попавших одновременно в группы A и B: {len(intersect_users)}")

Количество пользователей, попавших одновременно в группы A и B: 0


Пересечений пользователей нет

### 3\.2 Анализ данных о пользовательской активности по таблице `ab_test_events`:

In [ ]:
# Оставляем только участников нужного теста
interface_users = participants_eu_clean[participants_eu_clean['ab_test'] == 'interface_eu_test']['user_id'].unique()

# Фильтруем события по этим пользователям
interface_events = events[events['user_id'].isin(interface_users)]

# Проверим результат
print(f"Всего событий: {len(interface_events)}")
print(interface_events.head())

Всего событий: 72935
                user_id            event_dt    event_name details
64672  5F506CEBEDC05D30 2020-12-06 14:10:01  registration     0.0
64946  51278A006E918D97 2020-12-06 14:37:25  registration    -3.8
66585  A0C1E8EFAD874D8B 2020-12-06 17:20:22  registration   -3.32
67873  275A8D6254ACF530 2020-12-06 19:36:54  registration   -0.48
67930  0B704EB2DC7FCA4B 2020-12-06 19:42:20  registration     0.0


In [ ]:
# Приводим дату события к нужному формату
interface_events['event_dt'] = pd.to_datetime(interface_events['event_dt'])

# Находим дату регистрации каждого пользователя — первое событие 'registration'
registration = interface_events[interface_events['event_name'] == 'registration'].groupby('user_id')['event_dt'].min().reset_index()
registration = registration.rename(columns={'event_dt': 'registration_dt'})

/tmp/ipykernel_164/14513223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interface_events['event_dt'] = pd.to_datetime(interface_events['event_dt'])


In [ ]:
# Объединяем исходный датафрейм с датами регистрации
interface_events = interface_events.merge(registration, on='user_id', how='left')

# Считаем, сколько дней прошло от регистрации до каждого события
interface_events['days_since_registration'] = (interface_events['event_dt'] - interface_events['registration_dt']).dt.days

# Оставляем только события в первые 7 дней (включительно)
events_7days = interface_events[(interface_events['days_since_registration'] >= 0) & (interface_events['days_since_registration'] <= 6)]

print(f"Событий в первые 7 дней после регистрации: {len(events_7days)}")

Событий в первые 7 дней после регистрации: 62934


In [ ]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Задаём параметры
baseline_conversion = 0.3  # 30%
mde = 0.03  # Минимальный детектируемый эффект — можно менять
alpha = 0.05  # Уровень значимости
power = 0.8  # Мощность теста

# Рассчитываем effect size из baseline и mde
effect_size = proportion_effectsize(baseline_conversion, baseline_conversion + mde)

# Инициализируем класс для анализа мощности теста
power_analysis = NormalIndPower()

# Рассчитываем размер выборки для каждой группы (A и B)
sample_size_per_group = power_analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1)

print(f"Минимальный размер выборки на каждую группу: {int(sample_size_per_group)} пользователей")

Минимальный размер выборки на каждую группу: 3761 пользователей


Если в каждой группе A и B фактически присутствует не меньше 3761 пользователя, то выборка считается достаточной для обнаружения статистически значимого изменения конверсии при условии корректного проведения теста и отсутствия других искажений.

Если размер групп меньше этого значения, то тест может не иметь достаточной мощности — существует риск пропуска реального эффекта (ошибка второго рода).

- рассчет для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [ ]:
# Фильтруем события покупки
purchases = events_7days[(events_7days['user_id'].isin(interface_users)) & (events_7days['event_name'] == 'purchase')]

# Добавляем информацию о группе
purchases = purchases.merge(participants_eu[['user_id', 'group']], on='user_id', how='left')

# Считаем количество уникальных покупателей по группам
buyers_per_group = purchases.groupby('group')['user_id'].nunique().reset_index(name='buyers')

# Считаем общее количество пользователей в каждой группе
total_per_group = participants_eu.groupby('group')['user_id'].nunique().reset_index(name='total_users')

# Объединяем результаты
summary = buyers_per_group.merge(total_per_group, on='group')

# Добавляем конверсию
summary['conversion'] = (summary['buyers'] / summary['total_users']).round(5)

print(summary)

  group  buyers  total_users  conversion
0     A    1480         5383     0.27494
1     B    1579         5467     0.28882


- Конверсия в тестовой группе B составила 28.88%, в то время как в контрольной группе A — 27.49%.
Разница в конверсии: 28.88% − 27.49% = 1.39 п.п.

Таким образом, в тестовой группе наблюдается небольшой рост конверсии по сравнению с контрольной. Однако:
  - прирост меньше целевого значения в 3 п.п., заданного в гипотезе;
  - перед принятием решения необходимо проверить статистическую значимость этой разницы.


## 4. Оценка результатов A/B-тестирования:

- Проверка изменения конверсии

In [ ]:
summary['conversion'] = summary['buyers'] / summary['total_users']

# Выделим конверсии по группам
conv_A = summary.loc[summary['group'] == 'A', 'conversion'].values[0]
conv_B = summary.loc[summary['group'] == 'B', 'conversion'].values[0]

print(f"Конверсия группы A: {conv_A:.4f}")
print(f"Конверсия группы B: {conv_B:.4f}")

Конверсия группы A: 0.2749
Конверсия группы B: 0.2888


Гипотеза:

- H0: Конверсия в группе B не выше, чем в группе A..
- H1: Конверсия в группе B выше, чем в группе A.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
# Сохраняем значения в нужном порядке: сначала B, потом A
successes = summary.sort_values('group', ascending=False)['buyers'].values
nobs = summary.sort_values('group', ascending=False)['total_users'].values

# Односторонний z-тест: альтернативная гипотеза — конверсия B > конверсии A
stat, p_value = proportions_ztest(successes, nobs, alternative='larger')

print(f"Z-статистика: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

Z-статистика: 1.6071
p-value: 0.0540


In [ ]:
# Односторонний тест (alternative='larger' означает H1: конверсия B > A)
stat, p_value = proportions_ztest(successes, nobs, alternative='larger')

print(f"Z-статистика: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("Отклоняем H0: Конверсия в группе B статистически значимо выше.")
else:
    print("Нет оснований отклонить H0: статистически значимых различий нет.")

Z-статистика: 1.6071
p-value: 0.0540
Нет оснований отклонить H0: статистически значимых различий нет.


Конверсия в группе A составила 27.49%, а в группе B — 28.88%.
Абсолютная разница — +1.39 п.п.

Z-тест на пропорции показал пограничное значение статистической значимости:
Z-статистика = 1.61, p-value = 0.0540, что немного превышает порог α = 0.05.

Таким образом:

- Оснований отклонить нулевую гипотезу нет — статистической значимости в пользу роста конверсии недостаточно.
- Целевой прирост в 3 п.п. не достигнут (фактически прирост 1.39 п.п.).
- Хотя наблюдается положительная динамика в тестовой группе, она неубедительна с точки зрения статистики и требований ТЗ.

Вывод: на данном этапе внедрение нового интерфейса нецелесообразно. Рекомендуется либо провести тест с большей выборкой или длительностью, либо пересмотреть изменения в интерфейсе, чтобы достичь более выраженного эффекта.